In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, roc_auc_score, roc_curve, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn import clone
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.exceptions import NotFittedError
import itertools
from pprint import pprint
from tempeh.configurations import datasets
# from fairlearn.widget import FairlearnDashboard
from fairlearn.reductions import GridSearch#, DemographicParity
from sklearn.calibration import CalibratedClassifierCV
from fairlearn.postprocessing import ThresholdOptimizer
%matplotlib inline

In [2]:
path = 'datasets/student-loan-approval.csv'
#csv_url = 'https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv'
df = pd.read_csv(path)
df.drop(columns=['Loan_ID'], inplace=True)
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             601 non-null    object 
 1   Married            611 non-null    object 
 2   Dependents         599 non-null    object 
 3   Education          614 non-null    object 
 4   Self_Employed      582 non-null    object 
 5   ApplicantIncome    614 non-null    int64  
 6   CoapplicantIncome  614 non-null    float64
 7   LoanAmount         592 non-null    float64
 8   Loan_Amount_Term   600 non-null    float64
 9   Credit_History     564 non-null    float64
 10  Property_Area      614 non-null    object 
 11  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(7)
memory usage: 57.7+ KB


In [4]:
df.isnull().sum()

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [5]:
len(df)

614

In [6]:
df = df[(~df['Gender'].isnull()) & (~df['Married'].isnull())]
dependents_dict = {'0': '0', '1': '1', '2': '2', '3+': '3'}
df['Dependents'] = df['Dependents'].map(dependents_dict)
df = df[(~df['LoanAmount'].isnull()) & (~df['Loan_Amount_Term'].isnull())]
fill_values = {'Self_Employed': 'NaN', 'Dependents': 'NaN', 'Credit_History': -1.0}
df.fillna(value=fill_values, inplace=True)

df.isnull().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [7]:
df['Credit_History'] = df.Credit_History.astype(object)
categorical_features = df.columns[df.dtypes==object].tolist() 
le = LabelEncoder()

df[categorical_features] = df[categorical_features].apply(lambda col: le.fit_transform(col))

# Apply OneHotEncoder on each of the categorical columns
categorical_cols = ['Self_Employed', 'Credit_History', 'Property_Area', 'Dependents']

encoded_features = []
ohe = OneHotEncoder()
for feature in categorical_cols:
    encoded_feat = OneHotEncoder(drop='first').fit_transform(df[feature].values.reshape(-1, 1)).toarray()
    n = df[feature].nunique()
    cols = ['{}_{}'.format(feature, n) for n in range(0, n-1)]
    encoded_df = pd.DataFrame(encoded_feat, columns=cols)
    encoded_df.index = df.index
    encoded_features.append(encoded_df)
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,1,1,1,0,1,4583,1508.0,128.0,360.0,2,0,0
2,1,1,0,0,2,3000,0.0,66.0,360.0,2,2,1
3,1,1,0,1,1,2583,2358.0,120.0,360.0,2,2,1
4,1,0,0,0,1,6000,0.0,141.0,360.0,2,2,1
5,1,1,2,0,2,5417,4196.0,267.0,360.0,2,2,1


In [8]:
female_denied = len(df[(df['Gender'] == 0) & (df['Loan_Status'] == 0)])
female_accepted = len(df[(df['Gender'] == 0) & (df['Loan_Status'] == 1)])

male_denied = len(df[(df['Gender'] == 1) & (df['Loan_Status'] == 0)])
male_accepted = len(df[(df['Gender'] == 1) & (df['Loan_Status'] == 1)])

male_approval_rate = male_accepted/(male_accepted + male_denied)
female_approval_rate = female_accepted/(female_accepted + female_denied)
print(f'Male Approval Rate: {male_approval_rate}\nFemale Approval Rate: {female_approval_rate}')
'''
Severe Data imbalance but similar data rates
'''

Male Approval Rate: 0.7045951859956237
Female Approval Rate: 0.6698113207547169


'\nSevere Data imbalance but similar data rates\n'

In [9]:
data = df.copy()
labels = data.pop('Loan_Status')
sensitive_attributes = data['Gender']

X_train, X_test, y_train, y_test, sensitive_attributes_train, sensitive_attributes_test = train_test_split(data, labels,
                                                                                                         sensitive_attributes,
                                                                                                         stratify=df['Loan_Status'], 
                                                                                                         test_size=0.25,
                                                                                                         random_state = 42)


In [10]:
merged_train = pd.concat([X_train, y_train], axis=1)
gender_grouped = merged_train.groupby('Gender')
#Check if data distribution in approval rates is similar to previous rates
counts_by_gender = gender_grouped[['Loan_Status']].count().rename(columns={'Loan_Status': 'Count'})
rates_by_gender = gender_grouped[['Loan_Status']].mean().rename(columns={'Loan_Status': 'Apprvoal_Rate'})
pd.concat([counts_by_gender, rates_by_gender], axis=1)

,Count,Apprvoal_Rate
Gender,,
0,82,0.682927
1,340,0.702941


In [11]:
#Training Logisitic Regression
fairness_unaware_lr = LogisticRegression(solver='liblinear', fit_intercept=True)
fairness_unaware_lr.fit(X_train, y_train)

y_preds_lr = fairness_unaware_lr.predict(X_test)
accuracy_score(y_test, y_preds_lr)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

0.6808510638297872

In [12]:
merged_test_lr = pd.concat([X_test, y_test], axis=1)
merged_test_lr['Loan_Status_Preds'] = y_preds_lr
merged_test_lr.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Loan_Status_Preds
64,0,0,0,0,1,4166,0.0,116.0,360.0,1,1,0,1
590,1,1,0,0,1,3000,3416.0,56.0,180.0,2,1,1,1
602,1,1,3,0,1,5703,0.0,128.0,360.0,2,2,1,1
195,1,1,1,0,1,3125,2583.0,170.0,360.0,2,1,0,1
271,1,1,0,0,1,11146,0.0,136.0,360.0,2,2,1,1


In [13]:
#Training Random Forest
fairness_unaware_rf = RandomForestClassifier(
                      n_jobs = -1,
                      random_state = 42,
                      max_features = 'auto')

fairness_unaware_rf.fit(X_train, y_train)
y_preds_rf = fairness_unaware_rf.predict(X_test)
accuracy_score(y_test, y_preds_rf)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

0.8581560283687943

In [14]:
merged_test_rf = pd.concat([X_test, y_test], axis=1)
merged_test_rf['Loan_Status_Preds'] = y_preds_rf
merged_test_rf.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Loan_Status_Preds
64,0,0,0,0,1,4166,0.0,116.0,360.0,1,1,0,0
590,1,1,0,0,1,3000,3416.0,56.0,180.0,2,1,1,1
602,1,1,3,0,1,5703,0.0,128.0,360.0,2,2,1,1
195,1,1,1,0,1,3125,2583.0,170.0,360.0,2,1,0,1
271,1,1,0,0,1,11146,0.0,136.0,360.0,2,2,1,1


In [15]:
# confusion_values = confusion_matrix(merged_test_lr['Loan_Status'], merged_test_lr['Loan_Status_Preds'])
# [[true_negatives , false_positives],[false_negatives , true_positives]] = confusion_values
# [[true_negatives , false_positives],[false_negatives , true_positives]]
# recall = true_positives/ (true_positives + false_negatives)
# precision = true_positives/ (true_positives + false_positives)
# recall
# precision

def calc_precision_recall(df):
    confusion_values = confusion_matrix(df['Loan_Status'], df['Loan_Status_Preds'])
    [[true_negatives , false_positives],[false_negatives , true_positives]] = confusion_values
    [[true_negatives , false_positives],[false_negatives , true_positives]]
    recall = true_positives/ (true_positives + false_negatives)
    precision = true_positives/ (true_positives + false_positives)
    return precision, recall

lr_precision, lr_recall = calc_precision_recall(merged_test_lr)
rf_precision, rf_recall = calc_precision_recall(merged_test_rf)

print(f'LR Precision, Recall:{lr_precision, lr_recall}\nRF Precision, Recall:{rf_precision, rf_recall}')

LR Precision, Recall:(0.7022900763358778, 0.9387755102040817)
RF Precision, Recall:(0.8545454545454545, 0.9591836734693877)


In [16]:
'''
Overall Accuracy Equality Using Logistic Regression
'''
(merged_test_lr['Loan_Status'] == merged_test_lr['Loan_Status_Preds']).astype(int).groupby(merged_test_lr['Gender']).mean()
'''
Overall Accuracy Equality Using Random Forest
'''
(merged_test_rf['Loan_Status'] == merged_test_rf['Loan_Status_Preds']).astype(int).groupby(merged_test_rf['Gender']).mean()

'\nOverall Accuracy Equality Using Logistic Regression\n'

Gender
0    0.583333
1    0.700855
dtype: float64

'\nOverall Accuracy Equality Using Random Forest\n'

Gender
0    0.916667
1    0.846154
dtype: float64

In [17]:
'''
checking probability for loan status for those who got their loan approved

Fairness Type: Predictive Parity Using Logistic Regression
'''
merged_test_lr[merged_test_lr['Loan_Status']==1]['Loan_Status_Preds'].groupby(merged_test_lr['Gender']).mean()

'''
checking probability for loan status for those who got their loan approved

Fairness Type: Predictive Parity Using Random Forest
'''
merged_test_rf[merged_test_rf['Loan_Status']==1]['Loan_Status_Preds'].groupby(merged_test_rf['Gender']).mean()

'\nchecking probability for loan status for those who got their loan approved\n\nFairness Type: Predictive Parity Using Logistic Regression\n'

Gender
0    0.933333
1    0.939759
Name: Loan_Status_Preds, dtype: float64

'\nchecking probability for loan status for those who got their loan approved\n\nFairness Type: Predictive Parity Using Random Forest\n'

Gender
0    1.000000
1    0.951807
Name: Loan_Status_Preds, dtype: float64

In [18]:
'''
Checking average Loan Approvals of both genders on LR
'''
merged_test_lr.groupby('Gender').agg({'Loan_Status': 'mean',  
                        'Loan_Status_Preds': 'mean'}).transpose()

'''
Checking average Loan Approvals of both genders on RF
'''
merged_test_rf.groupby('Gender').agg({'Loan_Status': 'mean',  
                        'Loan_Status_Preds': 'mean'}).transpose()

'\nChecking average Loan Approvals of both genders on LR\n'

Gender,0,1
Loan_Status,0.625000,0.709402
Loan_Status_Preds,0.958333,0.923077


'\nChecking average Loan Approvals of both genders on RF\n'

Gender,0,1
Loan_Status,0.625000,0.709402
Loan_Status_Preds,0.708333,0.794872


In [19]:
# male_df = merged_test_lr[merged_test_lr['Gender'] == 1]
# female_df = merged_test_lr[merged_test_lr['Gender'] == 0]


# male_confusion_values = confusion_matrix(male_df['Loan_Status'], male_df['Loan_Status_Preds'])
# [[male_tn , male_fp],[male_fn , male_tp]] = male_confusion_values

# female_confusion_values = confusion_matrix(female_df['Loan_Status'], female_df['Loan_Status_Preds'])
# [[female_tn , female_fp],[female_fn , female_tp]] = female_confusion_values
# [[male_tn , male_fp],[male_fn , male_tp]]
# [[female_tn , female_fp],[female_fn , female_tp]]

def calc_confusion_metrics(df, tag=False):
    male_df = df[df['Gender'] == 1]
    female_df = df[df['Gender'] == 0]

    pred_col = 'Loan_Status_Fair' if tag else 'Loan_Status_Preds'
        
    male_confusion_values = confusion_matrix(male_df['Loan_Status'], male_df[pred_col])
    [[male_tn , male_fp],[male_fn , male_tp]] = male_confusion_values

    female_confusion_values = confusion_matrix(female_df['Loan_Status'], female_df[pred_col])
    [[female_tn , female_fp],[female_fn , female_tp]] = female_confusion_values
    return [[male_tn , male_fp],[male_fn , male_tp]],[[female_tn , female_fp],[female_fn , female_tp]]

lr_male_metrics, lr_female_metrics = calc_confusion_metrics(merged_test_lr)
rf_male_metrics, rf_female_metrics = calc_confusion_metrics(merged_test_rf)
# [[white_tn , white_fp],[white_fn , white_tp]] = 
# [[white_tn , white_fp],[white_fn , white_tp]]

print(f'LR CM for M: {lr_male_metrics}\tF: {lr_female_metrics}')
print(f'RF CM for M: {rf_male_metrics}\tF: {rf_female_metrics}')

LR CM for M: [[4, 30], [5, 78]]	F: [[0, 9], [1, 14]]
RF CM for M: [[20, 14], [4, 79]]	F: [[7, 2], [0, 15]]


In [20]:
def fpr_fnr(male_metrics,female_metrics, tag= 'LR'):
    [[male_tn , male_fp],[male_fn , male_tp]] = male_metrics
    [[female_tn , female_fp],[female_fn , female_tp]] = female_metrics
    
    female_fpr = female_fp/ (female_fp + female_tn)
    male_fpr = male_fp/ (male_fp + male_tn)

    female_fnr = female_fn/ (female_fn + female_tp)
    male_fnr = male_fn/ (male_fn + male_tp)
    print(tag)
    print(f'FPR for Female Gender: {female_fpr} | FPR for Male Gender: {male_fpr}')
    print(f'FNR for Female Gender: {female_fnr} | FNR for Male Gender: {male_fnr}\n\n')
    
fpr_fnr(lr_male_metrics, lr_female_metrics, tag= 'LR')
fpr_fnr(rf_male_metrics, rf_female_metrics, tag= 'RF')

LR
FPR for Female Gender: 1.0 | FPR for Male Gender: 0.8823529411764706
FNR for Female Gender: 0.06666666666666667 | FNR for Male Gender: 0.060240963855421686


RF
FPR for Female Gender: 0.2222222222222222 | FPR for Male Gender: 0.4117647058823529
FNR for Female Gender: 0.0 | FNR for Male Gender: 0.04819277108433735




# Mitigating Equalised Odds Unfairness

In [21]:
class RF(BaseEstimator, ClassifierMixin):
    def __init__(self, model):
        self.rf_model = model

    def fit(self, X, y):
        try:
            check_is_fitted(self.rf_model)
            self.rf_model_ = self.rf_model
        except NotFittedError:
            self.rf_model_ = clone(
                self.rf_model
            ).fit(X, y)
        return self

    def predict(self, X):
        scores = self.rf_model_.predict_proba(X)[:, 1]
        return scores
    
class LR(BaseEstimator, ClassifierMixin):
    def __init__(self, model):
        self.rf_model = model

    def fit(self, X, y):
        try:
            check_is_fitted(self.rf_model)
            self.rf_model_ = self.rf_model
        except NotFittedError:
            self.rf_model_ = clone(
                self.rf_model
            ).fit(X, y)
        return self

    def predict(self, X):
        scores = self.rf_model_.predict_proba(X)[:, 1]
        return scores

In [22]:
'''
First Unfairness Mitigation in  Random Forest Classifier
'''
estimator_wrapper_rf = RF(fairness_unaware_rf).fit(X_train, y_train)

postprocessed_predictor_EO_rf = ThresholdOptimizer(
    estimator=estimator_wrapper_rf, constraints="equalized_odds", prefit=True
    #estimator=estimator_wrapper_rf, constraints="false_positive_rate_parity", prefit=True
)

postprocessed_predictor_EO_rf.fit(
    X_train, y_train, sensitive_features=sensitive_attributes_train
)

fairness_aware_predictions_EO_train_rf = postprocessed_predictor_EO_rf.predict(
    X_train, sensitive_features=sensitive_attributes_train
)
fairness_aware_predictions_EO_test_rf = postprocessed_predictor_EO_rf.predict(
    X_test, sensitive_features=sensitive_attributes_test
)

'\nFirst Unfairness Mitigation in  Random Forest Classifier\n'

'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.


ThresholdOptimizer(constraints='equalized_odds', estimator=RF(model=None),
                   flip=False, grid_size=1000, objective='accuracy_score',
                   predict_method='deprecated', prefit=True)

In [23]:
unaware_y_pred = fairness_unaware_rf.predict(X_test)
accuracy_score(y_test, unaware_y_pred)

0.8581560283687943

In [24]:
accuracy_score(y_test, fairness_aware_predictions_EO_test_rf)

0.8439716312056738

In [25]:
merged_test_rf['Loan_Status_Fair'] =  fairness_aware_predictions_EO_test_rf
'''
Before Fairness Constraint Applied
'''
(merged_test_rf['Loan_Status'] == merged_test_rf['Loan_Status_Preds']).astype(int).groupby(merged_test_rf['Gender']).mean()
'''
After Fairness Constraint Applied. Slight Decrease for males
'''
(merged_test_rf['Loan_Status'] == merged_test_rf['Loan_Status_Fair']).astype(int).groupby(merged_test_rf['Gender']).mean()

'\nBefore Fairness Constraint Applied\n'

Gender
0    0.916667
1    0.846154
dtype: float64

'\nAfter Fairness Constraint Applied. Slight Decrease for males\n'

Gender
0    0.916667
1    0.829060
dtype: float64

In [26]:
'''
checking probability for loan status for those who got their loan approved

Fairness Type: Predictive Parity Using Random Forest
'''
merged_test_rf[merged_test_rf['Loan_Status']==1]['Loan_Status_Preds'].groupby(merged_test_rf['Gender']).mean()
'''
After FPR Parity solved. Dip in fairness
'''
merged_test_rf[merged_test_rf['Loan_Status']==1]['Loan_Status_Fair'].groupby(merged_test_rf['Gender']).mean()

'\nchecking probability for loan status for those who got their loan approved\n\nFairness Type: Predictive Parity Using Random Forest\n'

Gender
0    1.000000
1    0.951807
Name: Loan_Status_Preds, dtype: float64

'\nAfter FPR Parity solved. Dip in fairness\n'

Gender
0    1.000000
1    0.927711
Name: Loan_Status_Fair, dtype: float64

In [27]:
'''
Checking average Loan Approvals of both genders on RF
'''
merged_test_rf.groupby('Gender').agg({'Loan_Status': 'mean',  
                        'Loan_Status_Preds': 'mean'}).transpose()
'''
Checking average Loan Approvals of both genders on RF After Fairness. Slight Improvement for males
'''
merged_test_rf.groupby('Gender').agg({'Loan_Status': 'mean',  
                        'Loan_Status_Fair': 'mean'}).transpose()

'\nChecking average Loan Approvals of both genders on RF\n'

Gender,0,1
Loan_Status,0.625000,0.709402
Loan_Status_Preds,0.708333,0.794872


'\nChecking average Loan Approvals of both genders on RF After Fairness. Slight Improvement for males\n'

Gender,0,1
Loan_Status,0.625000,0.709402
Loan_Status_Fair,0.708333,0.777778


In [28]:
rf_male_metrics, rf_female_metrics = calc_confusion_metrics(merged_test_rf, tag = False)
fpr_fnr(rf_male_metrics, rf_female_metrics, tag= 'RF')
'''
After fairness
'''
rf_male_metrics, rf_female_metrics = calc_confusion_metrics(merged_test_rf, tag = True)
fpr_fnr(rf_male_metrics, rf_female_metrics, tag= 'RF')

RF
FPR for Female Gender: 0.2222222222222222 | FPR for Male Gender: 0.4117647058823529
FNR for Female Gender: 0.0 | FNR for Male Gender: 0.04819277108433735




'\nAfter fairness\n'

RF
FPR for Female Gender: 0.2222222222222222 | FPR for Male Gender: 0.4117647058823529
FNR for Female Gender: 0.0 | FNR for Male Gender: 0.07228915662650602




# Mitigating Logistic Regression Unfairness

In [29]:
'''
First Unfairness Mitigation in  Logistic Regression Classifier
'''
estimator_wrapper_lr = LR(fairness_unaware_lr).fit(X_train, y_train)

postprocessed_predictor_EO_lr = ThresholdOptimizer(
    estimator=estimator_wrapper_lr, constraints="equalized_odds", prefit=True
    #estimator=estimator_wrapper_rf, constraints="false_positive_rate_parity", prefit=True
)

postprocessed_predictor_EO_lr.fit(
    X_train, y_train, sensitive_features=sensitive_attributes_train
)

fairness_aware_predictions_EO_train_lr = postprocessed_predictor_EO_lr.predict(
    X_train, sensitive_features=sensitive_attributes_train
)
fairness_aware_predictions_EO_test_lr = postprocessed_predictor_EO_lr.predict(
    X_test, sensitive_features=sensitive_attributes_test
)
unaware_y_pred_lr = fairness_unaware_lr.predict(X_test)
accuracy_score(y_test, unaware_y_pred_lr)

'\nFirst Unfairness Mitigation in  Logistic Regression Classifier\n'

'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.


ThresholdOptimizer(constraints='equalized_odds', estimator=LR(model=None),
                   flip=False, grid_size=1000, objective='accuracy_score',
                   predict_method='deprecated', prefit=True)

0.6808510638297872

In [30]:
accuracy_score(y_test, fairness_aware_predictions_EO_test_lr)

0.7163120567375887

In [31]:
merged_test_lr['Loan_Status_Fair'] =  fairness_aware_predictions_EO_test_lr
'''
Before Fairness Constraint Applied
'''
(merged_test_lr['Loan_Status'] == merged_test_lr['Loan_Status_Preds']).astype(int).groupby(merged_test_lr['Gender']).mean()
'''
After Fairness Constraint Applied. Significant Increase
'''
(merged_test_lr['Loan_Status'] == merged_test_lr['Loan_Status_Fair']).astype(int).groupby(merged_test_lr['Gender']).mean()

'\nBefore Fairness Constraint Applied\n'

Gender
0    0.583333
1    0.700855
dtype: float64

'\nAfter Fairness Constraint Applied. Significant Increase\n'

Gender
0    0.625000
1    0.735043
dtype: float64

In [32]:
'''
checking probability for loan status for those who got their loan approved

Fairness Type: Predictive Parity Using LR
'''
merged_test_lr[merged_test_lr['Loan_Status']==1]['Loan_Status_Preds'].groupby(merged_test_lr['Gender']).mean()
'''
After FPR Parity solved. Significant Improvement.
'''
merged_test_lr[merged_test_lr['Loan_Status']==1]['Loan_Status_Fair'].groupby(merged_test_lr['Gender']).mean()

'\nchecking probability for loan status for those who got their loan approved\n\nFairness Type: Predictive Parity Using LR\n'

Gender
0    0.933333
1    0.939759
Name: Loan_Status_Preds, dtype: float64

'\nAfter FPR Parity solved. Significant Improvement.\n'

Gender
0    1
1    1
Name: Loan_Status_Fair, dtype: int64

In [33]:
'''
Checking average Loan Approvals of both genders on LR
'''
merged_test_lr.groupby('Gender').agg({'Loan_Status': 'mean',  
                        'Loan_Status_Preds': 'mean'}).transpose()
'''
Checking average Loan Approvals of both genders on RF After Fairness. Slight Improvement for males
'''
merged_test_lr.groupby('Gender').agg({'Loan_Status': 'mean',  
                        'Loan_Status_Fair': 'mean'}).transpose()

'\nChecking average Loan Approvals of both genders on LR\n'

Gender,0,1
Loan_Status,0.625000,0.709402
Loan_Status_Preds,0.958333,0.923077


'\nChecking average Loan Approvals of both genders on RF After Fairness. Slight Improvement for males\n'

Gender,0,1
Loan_Status,0.625,0.709402
Loan_Status_Fair,1.000,0.974359


In [34]:
'''
Equalised Odds
'''
lr_male_metrics, lr_female_metrics = calc_confusion_metrics(merged_test_lr, tag = False)
fpr_fnr(lr_male_metrics, lr_female_metrics, tag= 'LR')
'''
After fairness
'''
lr_male_metrics, lr_female_metrics = calc_confusion_metrics(merged_test_lr, tag = True)
fpr_fnr(lr_male_metrics, lr_female_metrics, tag= 'LR')

'\nEqualised Odds\n'

LR
FPR for Female Gender: 1.0 | FPR for Male Gender: 0.8823529411764706
FNR for Female Gender: 0.06666666666666667 | FNR for Male Gender: 0.060240963855421686




'\nAfter fairness\n'

LR
FPR for Female Gender: 1.0 | FPR for Male Gender: 0.9117647058823529
FNR for Female Gender: 0.0 | FNR for Male Gender: 0.0




In [41]:
def show_proportions(
    X, sensitive_features, y_pred, y=None, description=None, plot_row_index=1
):
    print("\n" + description)
    plt.figure(plot_row_index)
    plt.title(description)
    plt.ylabel("P[recidivism predicted | conditions]")

    indices = {}
    positive_indices = {}
    negative_indices = {}
    recidivism_count = {}
    recidivism_pct = {}
    groups = np.unique(sensitive_features.values)
    n_groups = len(groups)
    max_group_length = 1#max([len(str(group)) for group in groups])
    color = cm.rainbow(np.linspace(0, 1, n_groups))
    x_tick_labels_basic = []
    x_tick_labels_by_label = []
    for index, group in enumerate(groups):
        indices[group] = sensitive_features.index[sensitive_features == group]
        recidivism_count[group] = sum(y_pred[indices[group]])
        recidivism_pct[group] = recidivism_count[group] / len(indices[group])
        print(
            "P[recidivism predicted | {}]                {}= {}".format(
                group, " " * (max_group_length - len(group)), recidivism_pct[group]
            )
        )

        plt.bar(index + 1, recidivism_pct[group], color=color[index])
        x_tick_labels_basic.append(group)

        if y is not None:
            positive_indices[group] = sensitive_features.index[
                (sensitive_features == group) & (y == 1)
            ]
            negative_indices[group] = sensitive_features.index[
                (sensitive_features == group) & (y == 0)
            ]
            prob_1 = sum(y_pred[positive_indices[group]]) / len(positive_indices[group])
            prob_0 = sum(y_pred[negative_indices[group]]) / len(negative_indices[group])
            print(
                "P[recidivism predicted | {}, recidivism]    {}= {}".format(
                    group, " " * (max_group_length - len(group)), prob_1
                )
            )
            print(
                "P[recidivism predicted | {}, no recidivism] {}= {}".format(
                    group, " " * (max_group_length - len(group)), prob_0
                )
            )

            plt.bar(n_groups + 1 + 2 * index, prob_1, color=color[index])
            plt.bar(n_groups + 2 + 2 * index, prob_0, color=color[index])
            x_tick_labels_by_label.extend(
                ["{} recidivism".format(group), "{} no recidivism".format(group)]
            )

    x_tick_labels = x_tick_labels_basic + x_tick_labels_by_label
    plt.xticks(
        range(1, len(x_tick_labels) + 1),
        x_tick_labels,
        rotation=45,
        horizontalalignment="right",
    )

In [49]:
#X_test = X_test.drop(['Gender'], axis = 1)
X_test['Gender'] = sensitive_attributes_test.replace({"Female": 0, "Male": 1}, inplace=True)
sensitive_attributes_test.replace({0: "Female", 1: "Male"}, inplace=True)
show_proportions(
    X_test,
    sensitive_attributes_test,
    fairness_unaware_lr.predict(X_test),
    y_test,
    description="Fairness Unaware LR",
    plot_row_index=1,
)
# show_proportions(
#     X_test,
#     sensitive_attributes_test,
#     fairness_aware_predictions_EO_test_lr,
#     y_test,
#     description="Equalized Odds Postprocessing Using Fairlearn",
#     plot_row_index=2,
# )
# plt.show()

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [68]:
len(merged_test_lr[merged_test_lr['Gender']==0])

24